Pregel
---

Pregel is ["the system that changed graph processing"](https://www.coursera.org/learn/graph-analytics/lecture/eVChr/pregel-the-system-that-changed-graph-processing). It's a *parallel-processing* system suitable for processing large graphs. It was described by several Google authors in a highly-cited 2010 paper ([Malewicz et al, 2010](http://dl.acm.org/citation.cfm?id=1807184)). The name *Pregel* is the name of the river that flows through Konigsberg (now Kaliningrad), the setting of the seminal graph theory problem, the Seven Bridges of Konigsberg.

Motivation
---

A lot of graph processing problems require using large matrix computations, such as matrix factorisation and matrix powers. For large graphs (large $n$ nodes), the size of a matrix is $n^2$, so this is a problem. Some graphs are also too large to represent on a single machine, regardless of what format we store them or what algorithms we want to run. Therefore, to keep scaling to larger graphs, we need to think about how to divide the work between multiple machines.

Batch synchronous parallel
---

The BSP paradigm (Valiant, 1990) is one approach to this, and *Pregel* is one implementation of BSP, with some extra ideas, and some extra engineering (eg fault tolerance). The basic idea in both BSP and Pregel, then, is *think like a vertex*. We have to rewrite our algorithm as if it was being executed by each vertex. Every vertex executes the same algorithm.

The model requires two main concepts: time-steps and messages. At every time-step (called *superstep*), the framework calls an *update* method on each vertex. The vertex reads any incoming messages from other vertices, runs its algorithm, and sends out messages. Messages can only go along the (directed) edges from a vertex to its neighbours. The framework is responsible for book-keeping the messages, the time-steps, and some other stuff. 

Given a lot of machines, we then allocate our graph's vertices approximately evenly between the machines. So, eg, we might have 1 million vertices per machine. We try to allocate in such a way that most of the edges are between vertices on the same machine -- so most messages go between vertices on the same machine -- so minimising inter-machine communication, which is much slower.




Threads and processes
---

Most modern computers have a CPU with multiple processors or *cores*. For example, my laptop is dual-core. The cores are in some ways independent. We get the most out of our machines when we write software to use all the cores at once: we say then that several tasks are running *in parallel*. Some types of software lend themselves naturally to this, and others don't. Some algorithms are *easy to parallelize*. (Another way to get the most out of the machine is run multiple programs at the same time.)

To be concrete: if we write (say) a for loop inside a for loop inside a for loop, with something like a multiplication in the inner loop, and each loop is repeating 1000 times, then this will chug away for a long time on *one processor*. The other processor will lie idle (actually probably doing some background operating system stuff, but not working on our for loop). 

To actually use more than one processor, we have to use a software framework designed for that. There are two main software concepts available to us for parallelisation: *threads* and *processes*. We won't go into the differences between them. The central concept is similar: a program starts up on a single processor, then calls some special functions to start up extra threads (or processes), and the operating system can then run them on other processors. The extra threads (or processes) can communicate with each other and the original in various ways. Then can also quit, and new ones can be started.

In Pregel, each machine runs multiple threads. In `pregel.py`, the single machine runs multiple threads. Each thread is responsible for running some vertices. The "main" program is responsible for book-keeping -- passing messages, etc.

`pregel.py`
---

A toy version of Pregel was implemented by Nielsen (https://github.com/mnielsen/Pregel). I have made a few small additions, and my version is available at https://github.com/jmmcd/Pregel. (This document lives there.) I'll describe the code as it exists in my repo, but be aware that most of it was written by Nielsen.

By "toy version", we mean that it illustrates (most of) the concepts, but it doesn't actually implement processing on multiple machines. It does implement parallel processing, using *threads*.

`pregel.py` provides two main things: a `Pregel` class, and a `Vertex` class. `Pregel` is the "main" piece of code. It does some initialisation, then it iterates: at each superstep, it passes messages, and then calls `Vertex.update` for each vertex. `Vertex` is a base class, which doesn't do much: the idea is that we write a subclass, inheriting from `Vertex`, specialising to whatever algorithm we actually want to run. In particular, we have to write the `update` method.

`pagerank.py`
---

The Pregel paper describes a few applications. The first one provided in the code is PageRank. Recall that mathematically, PageRank is described as a generalised eigenvector problem, which can be solved using power iteration or other methods. The initial idea is that a vertex $i$ has PageRank $r_i$ described by the sum of the PageRanks of vertices that point to $i$, but each one divided by its number of outgoing links:

$$r_i = \sum_{\mathrm{edges} \ ji} {r_j \over d_j}$$

The "damping" factor $\alpha$ changes it slightly:

$$r_i = (1-\alpha){1 \over n} + \alpha \sum_{\mathrm{edges} \ ji} {r_j \over d_j}$$


Here, we solve it using Pregel. The part we need to understand is `PageRankVertex.update`. Remember, we have to "think like a vertex". At each superstep, "we" will receive a message from each vertex that links to us. Each message is of the format `(vertex, 1, pagerank)`. Here, `vertex` is a vertex id, and `1` is an edge weight (in PageRank, all edges are unweighted, so we'll use `1`). `pagerank` is that vertex's estimation of its own pagerank, *divided by* that vertex's number of outgoing edges. In other words, it is a current estimate of ${r_j \over d_j}$.

We sum over these incoming pagerank values and plug in to the formula to get an updated estimate for our own pagerank, which we store as `self.value` -- this variable is often used in `pregel.py` to store the current "state" of a vertex. 

We then send out messages along our outgoing edges, in the same format as before: `(vertex, 1, pagerank)`, where `vertex` is our id, `1` is the edge weight, and `pagerank` is our current estimate, divided by our number of outgoing edges.

All this happens for 50 supersteps. After that time, a node can (in Pregel terminology) "vote to halt", implemented as `self.active = False`.

We should observe that each vertex has only interacted with a small subset of the other vertices. No vertex is required to store all edges. There will be a lot of overhead in the communication here, but for many problems it will be a worthwhile trade-off.

`pagerank.py` runs Pregel/PageRank on a small randomly-generated graph. It also computes PageRank on the same graph using a linear algebra method, as a test. The result is pretty good, demonstrated when we see that the error between the vectors resulting from the `pregel.py` and the linear algebra implementations is small.


Exercises
---

1. Note that `pagerank.py` will divide by zero if any vertex has no outgoing edges. Think of a suitable fix for this.

2. Change the size of the random graph and/or the number of edges, to see whether it changes the error.

3. Try plugging in a NetworkX random graph generator (Erdos-Renyi, Watts-Strogatz, or Barabasi-Albert), instead of the simple graph generator used in `create_graph`, and again observing the effect on the error.

4. Extract $\alpha$ as a parameter `alpha = 0.85`, and remove the "hard-coded" constants 0.15 and 0.85. 

5. Then change $\alpha$ to different values (including extreme values 0 and 1) to see the effect on PageRank.

6. For (1) and (4), send me a pull request on Github.
